In [2]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [3]:
def readdata(datafile):
  
    with open(datafile, 'r') as file: linhas = file.readlines()

    # remove linha vazia inicial e elimina os "\n" de cada linha
    linhas = [a.strip() for a in linhas] 

    # lendo o tamanho da instancia
    aux = 0
    N = int(linhas[aux])
  
    # definindo vetores
    H = np.zeros(N)
    P = np.zeros(N)
    F = np.zeros(N)
    D = np.zeros(N)
  
    # lendo e armazenando dados
    aux = aux+1
    F[0] = float(linhas[aux])
    for i in range(1,N):
        F[i] = F[0]

    aux = aux+1
    H[0] = float(linhas[aux])
    for i in range(1,N):
        H[i] = H[0]

    aux = aux+1
    P[0] = float(linhas[aux])
    for i in range(1,N):
        P[i] = P[0]

    aux = aux+1
    D = np.fromstring(linhas[aux], dtype=float, sep = ' ')
 
    return N, H, P, F, D

In [4]:
def modelo(N, H, P, F, D, model):

    # adicionando variaveis
    x = model.addVars(N, name='x') 
    s = model.addVars(N, name='s')  
    y = model.addVars(N, vtype=GRB.BINARY, name='y') 

    # definindo funcao objetivo
    obj = 0
    for i in range(0, N):
        obj += P[i] * x[i]
        obj += H[i] * s[i]
        obj += F[i] * y[i]

    model.setObjective(obj, GRB.MINIMIZE)
  
    # definindo restrições
    model.addConstr(x[0] - s[0] == D[0])
    for i in range(1, N):
        model.addConstr(s[i-1] + x[i] - s[i] == D[i])
  
    for i in range(0, N):
        model.addConstr(x[i] - (D[i:N].sum())*y[i] <= 0)

    model.addConstr(s[N-1] == 0)

    # exportando modelo
    model.write("uls.lp")

    return model

In [5]:
def solve(model):
    # resolvendo o problema
    model.optimize()

In [9]:
if __name__ == "__main__":
    
    datafile = "../data/uls/52_2.txt"

    # lendo os dados
    N, H, P, F, D = readdata(datafile)

    # criando o modelo
    model = gp.Model("uls")

    # configurando parametros
    #model.Params.IterationLimit = 1000 # define o número de iterações do simplex
    model.Params.TimeLimit = 120 # define tempo limite
    model.Params.method = 0 #-1=automatic, 0=primal, 1=dual , 2=barrier
    #model.Params.NodeMethod = -1 #-1=automatic, 0=primal, 1=dual , 2=barrier
    model.Params.Threads = 1

    # modelando o problema
    modelo(N, H, P, F, D, model)

    # resolvendo o problema
    solve(model)
    

Set parameter TimeLimit to value 120
Set parameter Method to value 0
Set parameter Threads to value 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 1 threads

Optimize a model with 105 rows, 156 columns and 260 nonzeros
Model fingerprint: 0x5482fc9f
Variable types: 104 continuous, 52 integer (52 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [2e-01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+01, 1e+02]
Found heuristic solution: objective 15552.000000
Presolve removed 4 rows and 4 columns
Presolve time: 0.00s
Presolved: 101 rows, 152 columns, 252 nonzeros
Variable types: 101 continuous, 51 integer (51 binary)

Root relaxation: objective 6.159820e+03, 124 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      

In [10]:
# imprimindo solucao
for var in model.getVars():
    print(f"{var.varName} = {var.x}")

x[0] = 408.0
x[1] = 0.0
x[2] = 0.0
x[3] = 0.0
x[4] = 421.9999999999999
x[5] = 0.0
x[6] = 0.0
x[7] = 0.0
x[8] = 402.0000000000043
x[9] = 0.0
x[10] = 0.0
x[11] = 6.279664039080355e-12
x[12] = 370.99999999998937
x[13] = 0.0
x[14] = 0.0
x[15] = 0.0
x[16] = 462.99999999999966
x[17] = 0.0
x[18] = 0.0
x[19] = 0.0
x[20] = 0.0
x[21] = 495.0
x[22] = 0.0
x[23] = 0.0
x[24] = 0.0
x[25] = 0.0
x[26] = 419.9999999999973
x[27] = 0.0
x[28] = 0.0
x[29] = 0.0
x[30] = 403.9999999999946
x[31] = 5.6843418860808015e-12
x[32] = 0.0
x[33] = 0.0
x[34] = 381.0
x[35] = 0.0
x[36] = 0.0
x[37] = 0.0
x[38] = 414.0
x[39] = 0.0
x[40] = 0.0
x[41] = 0.0
x[42] = 521.9999999999993
x[43] = 0.0
x[44] = 0.0
x[45] = 0.0
x[46] = 0.0
x[47] = 450.0
x[48] = 0.0
x[49] = 0.0
x[50] = 0.0
x[51] = 0.0
s[0] = 304.0
s[1] = 196.0
s[2] = 108.0
s[3] = 0.0
s[4] = 316.9999999999999
s[5] = 219.99999999999991
s[6] = 112.99999999999991
s[7] = 0.0
s[8] = 302.0000000000043
s[9] = 195.00000000000435
s[10] = 102.00000000000435
s[11] = 1.0629719326971

In [11]:
# imprimindo o valor otimo
print(f"Objetivo = {model.objVal}")

Objetivo = 9263.399999999998
